In [42]:
! cat data/temperature/*.csv | awk -f com/temperature/removeEmpty.awk | awk -f com/temperature/replaceYearDate.awk | awk -f com/temperature/filterYears.awk | awk -f com/temperature/groupByYearAverageSum.awk  > result/temperature/result.csv

In [41]:
! awk -f com/energy/merge.awk data/energy/*.csv | awk -f com/energy/filterYears.awk  > result/energy/merged.csv

1995,Algeria
1995,Afghanistan
1996,Algeria
1996,Afghanistan
1997,Algeria
1997,Afghanistan
1998,Algeria
1998,Afghanistan
1999,Algeria
1999,Afghanistan
2000,Algeria
1990,Africa
2000,Afghanistan
2001,Algeria
1991,Africa
2001,Afghanistan
2002,Algeria
1992,Africa
2002,Afghanistan
2003,Algeria
2003,Afghanistan
1993,Africa
2004,Algeria
2004,Afghanistan
1994,Africa
2005,Algeria
2000,Africa
2005,Afghanistan
1995,Africa
2006,Algeria
2001,Africa
2006,Afghanistan
1996,Africa
2007,Algeria
1990,Albania
2002,Africa
1997,Africa
2007,Afghanistan
2020,Afghanistan
2008,Algeria
1991,Albania
2003,Africa
1998,Africa
2008,Afghanistan
2009,Algeria
1992,Albania
2004,Africa
1999,Africa
2009,Afghanistan
1993,Albania
2005,Africa
1994,Albania
2006,Africa
1995,Albania
2020,Africa
2007,Africa
1996,Albania
2008,Africa
1997,Albania
2009,Africa
1998,Albania
1999,Albania
2000,Albania
2001,Albania
2002,Albania
2003,Albania
2004,Albania
2005,Albania
2006,Albania
2020,Albania
2007,Albania
2008,Albania
2009,Albania
2010,Alg